

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">This is the experiment to analyze the stocks based on the Benjamin Graham's The Intelligent Investor. This tool Analyze any stock symbol from the NSE (National Stock Exchange) or BSE (Bombay Stock Exchange) 
This is just the learning purpose and no investment advice should be taken from this.
            </span>
        </td>
    </tr>
</table>

In [ ]:

from dotenv import load_dotenv

load_dotenv(override=True)

!uv add yfinance pandas




In [ ]:
# Install dependencies (only needed once in your env)
!uv add yfinance pandas

# Import them
import yfinance as yf
import pandas as pd

from IPython.display import Markdown, display

In [ ]:
# Example: Reliance Industries on NSE
stock_symbol = "TCS.NS"  # You can change to TCS.NS, INFY.NS, etc.

In [ ]:
# LLM client setup (OpenAI + Groq via OpenAI-compatible API)
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None
# Groq uses OpenAI-compatible endpoint|
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_client = OpenAI(api_key=GROQ_API_KEY, base_url=GROQ_BASE_URL) if GROQ_API_KEY else None

print(
    f"OpenAI: {'ON' if openai_client else 'OFF'} | Groq: {'ON' if groq_client else 'OFF'}"
)


In [ ]:
# Helper: compute Benjamin Graham-style checks (simplified for public data)
# Note: True Graham analysis needs per-share earnings for multi-year periods and balance-sheet figures.
# We use yfinance fields as proxies and document assumptions.
import numpy as np


def compute_graham_checks(ticker: str):
    T = yf.Ticker(ticker)

    # Price and trailing PE proxy
    info = T.info if hasattr(T, "info") else {}
    current_price = info.get("currentPrice")
    trailing_pe = info.get("trailingPE")
    forward_pe = info.get("forwardPE")
    peg_ratio = info.get("pegRatio")
    dividend_yield = info.get("dividendYield")  # fraction
    roe = info.get("returnOnEquity")  # fraction
    debt_to_equity = info.get("debtToEquity")  # percent
    current_ratio = info.get("currentRatio")
    book_value = info.get("bookValue")  # per share
    price_to_book = info.get("priceToBook")
    profit_margins = info.get("profitMargins")  # fraction

    # Conservative thresholds inspired by Graham (adjusted, simplified):
    # - PE <= 15 (or <= 20 if growth)  
    # - PB <= 1.5 (or PE*PB <= 22.5)  
    # - Dividend present preferred  
    # - Current ratio >= 1.5 for industrials (approx)  
    # - D/E <= 1 (approx)  
    # - Stable profitability (we proxy using positive margin and ROE)

    pe_ok = trailing_pe is not None and trailing_pe <= 15
    pb_ok = price_to_book is not None and price_to_book <= 1.5

    combo_ok = False
    if (trailing_pe is not None) and (price_to_book is not None):
        combo_ok = (trailing_pe * price_to_book) <= 22.5

    de_ok = (debt_to_equity is not None) and (debt_to_equity <= 100)  # percent
    cr_ok = (current_ratio is not None) and (current_ratio >= 1.5)
    div_ok = (dividend_yield is not None) and (dividend_yield > 0)
    profitability_ok = (
        (profit_margins is not None and profit_margins > 0)
        and (roe is not None and roe > 0)
    )

    checks = {
        "price": current_price,
        "trailing_pe": trailing_pe,
        "price_to_book": price_to_book,
        "pe_ok": pe_ok,
        "pb_ok": pb_ok,
        "pe_x_pb_ok": combo_ok,
        "dividend_yield": dividend_yield,
        "dividend_ok": div_ok,
        "debt_to_equity_pct": debt_to_equity,
        "debt_to_equity_ok": de_ok,
        "current_ratio": current_ratio,
        "current_ratio_ok": cr_ok,
        "profit_margins": profit_margins,
        "roe": roe,
        "profitability_ok": profitability_ok,
    }

    total_pass = sum(
        1
        for k in [
            "pe_ok",
            "pb_ok",
            "pe_x_pb_ok",
            "dividend_ok",
            "debt_to_equity_ok",
            "current_ratio_ok",
            "profitability_ok",
        ]
        if checks[k]
    )
    checks["passes"] = total_pass
    checks["total_criteria"] = 7
    return checks


def format_checks_md(ticker: str, checks: dict) -> str:
    yn = lambda b: "✅" if b else "❌"
    lines = [f"### Graham-style screen for `{ticker}`"]
    lines.append("- Price: " + str(checks.get("price")))
    lines.append(
        f"- PE: {checks.get('trailing_pe')} | PB: {checks.get('price_to_book')} | PE×PB<=22.5: {yn(checks.get('pe_x_pb_ok'))}"
    )
    lines.append(f"- PE<=15: {yn(checks.get('pe_ok'))}")
    lines.append(f"- PB<=1.5: {yn(checks.get('pb_ok'))}")
    lines.append(
        f"- Dividend yield: {checks.get('dividend_yield')} | Present: {yn(checks.get('dividend_ok'))}"
    )
    lines.append(
        f"- D/E%: {checks.get('debt_to_equity_pct')} | <=100%: {yn(checks.get('debt_to_equity_ok'))}"
    )
    lines.append(
        f"- Current ratio: {checks.get('current_ratio')} | >=1.5: {yn(checks.get('current_ratio_ok'))}"
    )
    lines.append(
        f"- Profit margin: {checks.get('profit_margins')} | ROE: {checks.get('roe')} | Positive: {yn(checks.get('profitability_ok'))}"
    )
    lines.append(
        f"- Score: {checks.get('passes')}/{checks.get('total_criteria')} criteria passed"
    )
    return "\n".join(lines)


In [ ]:
# Run metrics and display
checks = compute_graham_checks(stock_symbol)
display(Markdown(format_checks_md(stock_symbol, checks)))


In [ ]:
# LLM analysis using available provider(s)

def analyze_with_llm(summary_markdown: str, provider: str = "auto",
                      openai_model: str = "gpt-4o-mini",
                      groq_model: str = "llama-3.3-70b-versatile") -> str:
    sys_msg = (
        "You are a conservative value-investing analyst using Benjamin Graham principles. "
        "Explain clearly which checks passed/failed, what that implies, and caveats about proxies. "
        "Be concise and avoid recommendations; this is educational only."
    )
    user_msg = (
        "Analyze the following Graham-style checklist for a stock. "
        "Summarize pass/fail, key drivers, and a short risk note.\n\n" + summary_markdown
    )

    def chat(client, model):
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": sys_msg},
                {"role": "user", "content": user_msg},
            ],
            temperature=0.4,
            max_tokens=400,
        )
        return resp.choices[0].message.content

    outputs = []
    if provider in ("auto", "openai") and openai_client:
        try:
            outputs.append(("OpenAI", chat(openai_client, openai_model)))
        except Exception as e:
            outputs.append(("OpenAI", f"Error: {e}"))

    if provider in ("auto", "groq") and groq_client:
        try:
            outputs.append(("Groq", chat(groq_client, groq_model)))
        except Exception as e:
            outputs.append(("Groq", f"Error: {e}"))

    if not outputs:
        return "No LLM providers available. Set OPENAI_API_KEY and/or GROQ_API_KEY."

    merged = []
    for name, out in outputs:
        merged.append(f"### {name} analysis\n\n" + (out or ""))
    return "\n\n---\n\n".join(merged)

analysis = analyze_with_llm(format_checks_md(stock_symbol, checks))
display(Markdown(analysis))
